In [13]:
import global_resources as gr
import CAPMlib.CAPM as capm
import os
import pandas as pd

# Sperate based on labels

In [14]:
labels = gr.read_and_return_pd_df(os.path.join(gr.global_paths['Output'], 'k=10_processed_data.csv'))
labels['ID'] = labels['ID'].astype(str).str.zfill(6)

Reading files from: 'D:\Important Files\Repositories\Quantitative-Investment-Algorithms\Output\k=10_processed_data.csv'


In [15]:
print(labels.shape)

(5257, 3)


In [16]:
label_dfs = capm.label_dfs(labels, label_column = 'Labels')

In [17]:
time_series_dfs = capm.dfs

In [18]:
dfs = capm.create_nested_dfs(label_dfs, time_series_dfs, 'ID')

  - Label: 0, Stock ID: 000002
  - Label: 0, Stock ID: 000006
  - Label: 0, Stock ID: 000011
  - Label: 0, Stock ID: 000016
  - Label: 0, Stock ID: 000030
  - Label: 0, Stock ID: 000031
  - Label: 0, Stock ID: 000032
  - Label: 0, Stock ID: 000034
  - Label: 0, Stock ID: 000036
  - Label: 0, Stock ID: 000039
  - ... and 5204 more


In [21]:
display(time_series_dfs['000001'])

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
7824,2024-01-02,000001,1778.36,1749.10,1783.23,1749.10,1158366,1.075742e+09,1.92,-1.65,-29.26,0.60
7825,2024-01-03,000001,1745.85,1747.48,1750.73,1739.35,733610,6.736736e+08,0.65,-0.09,-1.62,0.38
7826,2024-01-04,000001,1745.85,1732.85,1745.85,1727.97,864194,7.874701e+08,1.02,-0.84,-14.63,0.45
7827,2024-01-05,000001,1731.22,1758.85,1786.48,1726.35,1991622,1.852660e+09,3.47,1.50,26.00,1.03
7828,2024-01-08,000001,1752.35,1739.35,1763.73,1732.85,1121156,1.029007e+09,1.76,-1.11,-19.50,0.58
...,...,...,...,...,...,...,...,...,...,...,...,...
8062,2024-12-26,000001,2346.39,2336.64,2348.02,2323.64,1000075,1.183746e+09,1.04,-0.42,-9.75,0.52
8063,2024-12-27,000001,2338.27,2331.76,2343.14,2304.13,1290012,1.518383e+09,1.67,-0.21,-4.88,0.66
8064,2024-12-30,000001,2323.64,2351.27,2354.52,2323.64,1351846,1.610892e+09,1.32,0.84,19.51,0.70
8065,2024-12-31,000001,2348.02,2310.64,2357.77,2310.64,1475367,1.747242e+09,2.00,-1.73,-40.63,0.76


In [54]:
result = {}
for label, stock_dfs in dfs.items():
    # Check if the cluster has any stocks
    if not stock_dfs:
        print(f"No stocks found in cluster {label}, skipping...")
        continue
        
    # Initialize a DataFrame with just the date column from the first stock
    first_stock_id = list(stock_dfs.keys())[0]
    time_series_df = pd.DataFrame({'日期': stock_dfs[first_stock_id]['日期']})
    
    # Collect percentage changes from all stocks in this cluster
    all_pct_changes = []
    for stock_id, df in stock_dfs.items():
        # Ensure alignment with the date column
        stock_pct = df.set_index('日期')['涨跌幅']
        all_pct_changes.append(stock_pct)
    
    # Combine all percentage changes and calculate the mean
    combined_pct = pd.concat(all_pct_changes, axis=1)
    avg_pct_change = combined_pct.mean(axis=1)
    
    # Join with the date column
    time_series_df = time_series_df.set_index('日期')
    time_series_df['Cluster Pct Change'] = avg_pct_change
    time_series_df = time_series_df.reset_index()
    
    # Store the result
    result[label] = time_series_df[['日期', 'Cluster Pct Change']]


No stocks found in cluster 5, skipping...
No stocks found in cluster 8, skipping...
No stocks found in cluster 9, skipping...


In [55]:
for key, df in result.items():
    print(key)
    display(df)

0


,日期,Cluster Pct Change
0,2024-01-02,-1.290
1,2024-01-03,-1.250
2,2024-01-04,-0.770
3,2024-01-05,0.170
4,2024-01-08,-1.340
...,...,...
238,2024-12-26,3.040
239,2024-12-27,0.210
240,2024-12-30,-1.060
241,2024-12-31,-3.445


1


,日期,Cluster Pct Change
0,2024-01-02,-0.750000
1,2024-01-03,-1.058571
2,2024-01-04,-0.375714
3,2024-01-05,-2.318571
4,2024-01-08,-2.301429
...,...,...
238,2024-12-26,2.548571
239,2024-12-27,-0.384286
240,2024-12-30,-0.762857
241,2024-12-31,-4.121429


2


...

,日期,Cluster Pct Change
0,2024-01-02,-0.84000
1,2024-01-03,-0.85125
2,2024-01-04,-0.41000
3,2024-01-05,-2.37500
4,2024-01-08,-2.32750
...,...,...
238,2024-12-26,1.05500
239,2024-12-27,0.78125
240,2024-12-30,-0.87750
241,2024-12-31,-2.14250


6


,日期,Cluster Pct Change
0,2024-01-02,2.35
1,2024-01-03,-0.43
2,2024-01-04,0.43
3,2024-01-05,-0.75
4,2024-01-08,-1.56
...,...,...
238,2024-12-26,3.65
239,2024-12-27,-1.02
240,2024-12-30,-11.35
241,2024-12-31,-1.98


7


,日期,Cluster Pct Change
0,2024-01-02,-0.578889
1,2024-01-03,-0.481111
2,2024-01-04,-0.954444
3,2024-01-05,-1.668889
4,2024-01-08,-2.541111
...,...,...
238,2024-12-26,4.262222
239,2024-12-27,-0.287778
240,2024-12-30,1.454444
241,2024-12-31,-3.898889


In [48]:
result = {}
for label, stock_dfs in dfs.items():
    time_series_df = pd.DataFrame()
    for stock_id, df in stock_dfs.items():
        if time_series_df.shape[1] == 0:
            time_series_df['日期'] = df['日期']
        df_reset = df['涨跌幅'].reset_index(drop=True)
        time_series_df = pd.concat([time_series_df, df_reset], axis = 1, ignore_index = True)
    
    cols = ['日期'] + list(stock_dfs.keys())
    time_series_df.columns = cols
    time_series_df['Cluster Pct Change'] = time_series_df.iloc[:, 1:].mean(axis = 1)
    result[label] = time_series_df['Cluster Pct Change']
    # display(time_series_df)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

In [32]:
for label, df in result.items():
    print(label)
    display(df)

0


0     -1.290
1     -1.250
2     -0.770
3      0.170
4     -1.340
       ...  
238    3.040
239    0.210
240   -1.060
241   -3.445
242   -2.955
Name: Cluster Pct Change, Length: 243, dtype: float64

1


0     -0.750000
1     -1.058571
2     -0.375714
3     -2.318571
4     -2.301429
         ...   
238    2.841667
239   -0.401667
240   -1.735000
241   -3.770000
242   -3.148333
Name: Cluster Pct Change, Length: 243, dtype: float64

2


...

0     -0.578889
1     -0.481111
2     -0.954444
3     -1.668889
4     -2.541111
         ...   
238    4.262222
239   -0.287778
240    1.454444
241   -3.898889
242   -2.483333
Name: Cluster Pct Change, Length: 243, dtype: float64

8


Series([], Name: Cluster Pct Change, dtype: float64)

9


Series([], Name: Cluster Pct Change, dtype: float64)